In [1]:
import sys
sys.path.append("../../")

In [ ]:
user_input = "Please extract the theme of the provided context in Maximum 5 words"

In [ ]:
from itops.storage.azure_blob.azure_blob_helper import AzureBlobHelper
from itops.storage.azure_blob.csv_helper import CSVHelper
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
import pandas as pd
from itops.db.mysql.mysqlhelper import MySQLHelper
from itops.config.configs import CONFIGS

In [ ]:
mysql_helper = MySQLHelper(CONFIGS.HOST,
                           CONFIGS.USERNAME_MYSQL,
                           CONFIGS.PASSWORD, "itops")

In [ ]:
print(CONFIGS.AZURE_BLOB_STORAGE_KEY)
print(CONFIGS.AZURE_BLOB_STORAGE_CONTAINER)
print(CONFIGS.AZURE_BLOB_STORAGE_ACCOUNT)

In [ ]:
filename = "ITSM_Data.csv"
num_clusters = 4  # Define the number of clusters

In [ ]:
azure_blob_helper = AzureBlobHelper(CONFIGS.AZURE_BLOB_STORAGE_ACCOUNT,
                                                      CONFIGS.AZURE_BLOB_STORAGE_KEY,
                                                      CONFIGS.AZURE_BLOB_STORAGE_CONTAINER)

In [ ]:
azure_blob_helper.list_blob()

In [ ]:
csv_helper = CSVHelper(azure_blob_helper)
df = csv_helper.read_file(filename)

In [ ]:
df.columns

In [ ]:
df.loc[1]["Text"]

In [ ]:
from itops.llm.azureopenaimanager.azure_open_ai_helper import AzureOpenAIManager

In [ ]:
azure_open_ai_helper = AzureOpenAIManager(endpoint=CONFIGS.AZURE_OPENAI_ENDPOINT,
                                          api_key =CONFIGS.AZURE_OPENAI_API_KEY,
                                          deployment_id=CONFIGS.AZURE_OPENAI_DEPLOYMENT_ID,
                    api_version="2023-05-15")

In [ ]:
content = df.iloc[15]["Text"]
print(content)
print(str(content))

In [ ]:
reply =azure_open_ai_helper.generate_reply_from_context(user_input, content, conversation=[])

In [ ]:
reply[0]

In [ ]:
reply_list = []
for i in range(len(df)):
    content = df.iloc[i]["Text"]
    content = str(content)
    reply = azure_open_ai_helper.generate_reply_from_context(user_input, 
                                                             content, 
                                                             conversation=[])
    reply_list.append(reply[0])
    reply =""
    print(f"Completed {i+1} ROW")

In [ ]:
df["themes"] = reply_list

In [ ]:
THEMES_FILE = filename.split(".")[0] +"_THEMES_ONLY" + "." + filename.split(".")[1]

In [ ]:
df.to_csv(THEMES_FILE,index = False)

In [ ]:
df_new = pd.read_csv(THEMES_FILE)

In [ ]:
df_new.columns

In [ ]:
df_new["themes"]

# Create Embeddings for the Themes

In [ ]:
MODEL_NAME = "all-MiniLM-L6-v2"

In [ ]:
def get_embedding_query_vector(query,model_name):
    """Get the vector of the query

    Args:
        query (string): user input

    Returns:
        _type_: vector of the query
    """
    model = SentenceTransformer(model_name)
    query_vector = model.encode(query)
    return query_vector

In [ ]:
embedding_list = []
df = df_new
for i in range(len(df)):
    content = df.iloc[i]["themes"]
    embedding = get_embedding_query_vector(content,MODEL_NAME)
    embedding_list.append(embedding)
    print(f"Completed {i+1} ROW")

In [ ]:
len(embedding_list)

In [ ]:
df["embedding"] = embedding_list

In [ ]:
df.head()

## Clustering of embeddings

In [ ]:

kmeans = KMeans(n_clusters=num_clusters, random_state=0)
kmeans.fit(embedding_list)

In [ ]:
labels = kmeans.labels_

In [ ]:
len(labels)

In [ ]:
df["CLUSTERS"] = labels

In [ ]:
df

In [ ]:
df["CLUSTERS"].value_counts()

In [ ]:
df[df["CLUSTERS"] == 0]

In [ ]:
cluster_name_list = []
for i in range(num_clusters):
    df_cluster = df[df["CLUSTERS"] == i]
    full_content = ""
    for j in range(len(df_cluster)):
        content = df_cluster.iloc[j]["Text"]
        full_content = full_content + str(content) + " \n "
    
    cluster_name = azure_open_ai_helper.generate_reply_from_context(user_input, 
                                                             full_content, 
                                                             conversation=[])
    cluster_name_list.append(cluster_name[0])
    cluster_name =""
    print(f"Completed {i+1} ROW")


In [ ]:
cluster_name_list

In [ ]:
df_cluster = pd.DataFrame()

In [ ]:
cluster_number_list

In [ ]:
cluster_name_list

In [ ]:
cluster_number_list= []
for i in range(len(cluster_name_list)):
    cluster_number_list.append(i)

In [ ]:
df_clusters = pd.DataFrame()

In [ ]:
df_clusters["CLUSTERS"] = cluster_number_list

In [ ]:
df_clusters

In [ ]:
df_clusters["CLUSTER_NAMES"] = cluster_name_list

In [ ]:
df.columns

In [ ]:
df_all = df.merge(df_clusters)

In [ ]:
df_all.columns

In [ ]:
RUN_NAME = "ITSM-RUN0"
NUMBER_OF_CLUSTERS = num_clusters 
CATEGORY = "ITSM"

In [ ]:
RUN_NAME

In [ ]:
FILENAME_INSIGHTS = filename.split(".")[0] +"-"+ RUN_NAME + "-"+ CATEGORY + "." + filename.split(".")[1]

In [ ]:
df_all.to_csv(FILENAME_INSIGHTS,index = False)

In [ ]:
azure_blob_helper.upload_blob_from_path(FILENAME_INSIGHTS,FILENAME_INSIGHTS)

In [ ]:
NUMBER_OF_CLUSTERS = num_clusters
CATEGORY = "ITSM"
INPUT_FILE_NAME_URL = f"{filename}"
FILENAME_INSIGHTS_URL = f"{FILENAME_INSIGHTS}"
SUB_CLUSTER_NAME =""
PARENT_CLUSTER_NAME = ""
NUMBER_OF_SUBCLUSTERS = ""

In [23]:
mysql_helper.connect()

Connected to MySQL database


In [ ]:
insert_query = """
    INSERT INTO run_log (RUN_NAME, SUB_CLUSTER_NAME, NUMBER_OF_CLUSTERS, PARENT_CLUSTER_NAME,
                            NUMBER_OF_SUBCLUSTERS, CATEGORY, INPUT_FILE_NAME, INSIGHTS_FILE_NAME,
                            CONTAINER_NAME,ACCOUNT_NAME)
    VALUES (%s,%s, %s, %s, %s, %s, %s, %s,%s, %s)
"""

data = (RUN_NAME, SUB_CLUSTER_NAME, NUMBER_OF_CLUSTERS, PARENT_CLUSTER_NAME,
                            NUMBER_OF_SUBCLUSTERS, CATEGORY, 
                            INPUT_FILE_NAME_URL, FILENAME_INSIGHTS_URL,
                            CONFIGS.AZURE_BLOB_STORAGE_CONTAINER,
                            CONFIGS.AZURE_BLOB_STORAGE_ACCOUNT)

In [26]:
mysql_helper.execute_query(insert_query, data)

0

In [28]:
# Fetch and display all records to verify insertion
select_query = "SELECT * FROM run_log WHERE RUN_NAME = %s "
run_name_to_search = RUN_NAME
records = mysql_helper.fetch_all(select_query,[run_name_to_search])

In [29]:
for record in records:
    print(record)

('ITSM-RUN0', '', '4', '', '', 'ITSM', 'https://stgtxtsql.blob.core.windows.net/itops/ITSM_Data.csv', 'https://stgtxtsql.blob.core.windows.net/itops/ITSM_Data-ITSM-RUN0-ITSM.csv', None, None)
('ITSM-RUN0', '', '4', '', '', 'ITSM', 'ITSM_Data.csv', 'ITSM_Data-ITSM-RUN0-ITSM.csv', 'itops', 'stgtxtsql')
